In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit

In [6]:
data = pd.read_csv('Data1\\Data1\\section 6\\avocado.csv', parse_dates=['Date'])

In [7]:
data.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


## training and testing data

### train all data before 2018

In [8]:
summr = data.groupby('Date')['Total Volume'].mean().reset_index()

In [9]:
summr.head()

,Date,Total Volume
0,2015-01-04,7.840216e+05
1,2015-01-11,7.273686e+05
2,2015-01-18,7.258221e+05
3,2015-01-25,7.080211e+05
4,2015-02-01,1.106048e+06


### renaming

In [11]:
#renaming the ts DataFrame's Columns
#ds for Date and y for response variable
renamed_s = summr.rename(columns={"Date": "ds", "Total Volume": "y"})
train_ts = renamed_s.loc[lambda df: df['ds'].dt.year < 2018, :]## before 2018
test_ts = renamed_s.loc[lambda df: df['ds'].dt.year == 2018, :]

In [13]:
from matplotlib.dates import MonthLocator, num2date
from matplotlib.ticker import FuncFormatter
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics

In [14]:
HORIZON = "90days"
PERIOD = "7days"

In [15]:
prophet_model = Prophet()
prophet_model.fit(train_ts)
## evaluate model performance
## pecify the forecast horizon
## optionally the size of the initial training period (initial)
prophet_cv_df = cross_validation(prophet_model, horizon=HORIZON, period=PERIOD)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 91 forecasts with cutoffs between 2016-01-11 00:00:00 and 2017-10-02 00:00:00


In [ ]:
## the performance metrics utility can be used to compute some useful statistics of the 
##pediction performance
##(yhat, yhat_lower, yhat_upper compared to y),
## as a function of the distance from the cutoff
## how far into the future the prediction was
prophet_cv_df.head()

In [ ]:
prophet_perf_df = performance_metrics(prophet_cv_df)

In [ ]:
prophet_perf_df.head()

## predict for the future

In [ ]:
future_prophet_df = prophet_model.make_future_dataframe(periods=365)
predict_prophet_df = prophet_model.predict(future_prophet_df)
predict_model.plot(predicted_prophet_df)
predict_model.plot_components(predicted_prophet_df)